In [ ]:
'''
For the main: 
1st have a function for control run
call it then run through autocorrelation time, etc. to get important inputs
Next have a function for the run, the first run takes the number of inputs and their parameters, then passes to RE algorithm
The RE algorithm saves metadata to a dataframe, then passes the last two columns to the run. 
So it is looped through to the end of the runs
Finally will look at some analysis, check PDFs, check ancestor trail
'''

In [2]:
import sys
import math
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

In [3]:
def get_initials():
    
    #control run initial parameterization
    control_run = pd.read_csv('control_run0505.csv')
    #remove transient
    control_dropped = control_run.loc[control_run['t']>=10]
    #access every 5th row (time unit)
    control_5th = control_dropped.iloc[::500]
    control_5th.head()
    cols = ['x','y','z','T', 'S']
    x_list, y_list, z_list, temp_list, salt_list = [control_5th[c].tolist() for c in cols]

    return x_list,y_list,z_list,temp_list,salt_list

In [4]:
#Gottwald model functions

def smoothabs(x, xi=10000):
    """Smooth absolute value function"""
    return x * np.tanh(x * xi)

def gottwald_noice(t, u, p, stochsys=True):
    """Gottwald model functions without sea ice"""
    x, y, z, T, S = u
    if stochsys:
        pvec = p[0] if isinstance(p, list) else p
    else:
        pvec = p
    
    (a, b, mu, epsilon_a, epsilon_f, F0, F1, G0, G1, 
     theta_0, theta_1, sigma_0, sigma_1, x_mean, Delta_mean) = pvec
    
    Delta = y**2 + z**2
    T_surf = theta_0 + theta_1 * (x - x_mean)/(np.sqrt(epsilon_f))
    S_surf = sigma_0 + sigma_1 * (Delta - Delta_mean)/(np.sqrt(epsilon_f))
    
    dx = 1/epsilon_f*(-Delta - a*(x - F0 - F1*T))
    dy = 1/epsilon_f*(x*y - b*x*z - (y - G0 + G1*T))
    dz = 1/epsilon_f*(b*x*y + x*z - z)
    dT = -1/epsilon_a*(T - T_surf) - T - mu*smoothabs(S-T)*T
    dS = S_surf - S - mu*smoothabs(S-T)*S
    
    return np.array([dx, dy, dz, dT, dS])

def param_gwn_default(param_l84=None, sigma_0=0.9): #**kwargs):
    """return parameter values"""
    if param_l84 is not None:
        a, b, F0, G0 = param_l84
    else:
        param_l84 = [0.25, 4., 8., 1.]  # default L84 params [a, b, F0, G0]
        
    x_std = 0.513
    Delta_std = 1.071
    
    x_mean = 1.0147
    Delta_mean = 1.7463  # long-run results from default L84
    
    # Unpack L84 params
    a, b, F0, G0 = param_l84
    
    # Other default model params
    mu = 7.5
    theta_0 = 1.
    # sigma_0 = 0.9  # Stommel params (is varied) 0.926, 0.932
    epsilon_a = 0.34
    epsilon_f = 0.0003  # 0.0083 # timescales
    F1 = 0.1
    G1 = 0.  # coupling params
    perturb_scaling = 0.01  # coupling strength of L84 to Stommel
    
    # Coupling params from L84 run/default
    # theta_1 = min(theta_0, perturb_scaling/x_std)
    # sigma_1 = min(sigma_0, perturb_scaling/Delta_std)

    theta_1=0.0195
    sigma_1=0.00934
    
    return [a, b, mu, epsilon_a, epsilon_f, F0, F1, G0, G1, 
            theta_0, theta_1, sigma_0, sigma_1, x_mean, Delta_mean]
    

def simulate_gottwald_noice(initial_conditions, params, t_span, t_eval=None, stochsys=True):
    """Simulate the Gottwald model without sea ice"""
    sol = solve_ivp(
        lambda t, y: gottwald_noice(t, y, params, stochsys),
        t_span,
        initial_conditions,
        t_eval=t_eval,
        method='RK45',
        rtol=1e-4,
        atol=1e-8
    )
    return sol

#main function for simulating, is called in the loop
def simulate_with_resampling(x_array,y_array,z_array,temp_array,salt_array):
    
    all_runs=[]
    matplotlib.use('Agg')
    
    # Get default Gottwald parameters
    params_gwn = param_gwn_default()
    print(f"Default Gottwald parameters:")
    print(params_gwn)

    # params_gwn_wrapped = [params_gwn]  # Wrap in list for stochsys format
    tmax = 1 # 00
    t_eval = np.linspace(0, tmax, tmax*100)
    amoc=np.zeros((2,2,tmax*100))
    
    # Simulate Gottwald model

    for temp in range(len(temp_array)):
        x0=x_array[temp]
        print('X0: ',x0,' Trajectory: ',temp)
        y0=y_array[temp]
        z0=z_array[temp]
        T0=temp_array[temp]
        S0=salt_array[temp]
        initial_gwn = [x0, y0, z0, T0, S0]  # [x, y, z, T, S]
            
        sol_gwn = simulate_gottwald_noice(initial_gwn, params_gwn, (0, tmax), 
                                       t_eval=t_eval, stochsys=False)
        data = {
            't': sol_gwn.t,
            'x': sol_gwn.y[0],
            'y': sol_gwn.y[1],
            'z': sol_gwn.y[2],
            'T': sol_gwn.y[3],
            'S': sol_gwn.y[4],
            'x0,y0,z0,T0,S0': f"{x0},{y0},{z0},{T0},{S0}",
            'AMOC': sol_gwn.y[3]-sol_gwn.y[4]
        }

        #print(amoc_df.head())
        
        #Convert this run to a dataframe
        run_df = pd.DataFrame(data)
    
        #Store it
        all_runs.append(run_df)

    #After the loop, combine everything:
    amoc_df = pd.concat(all_runs, ignore_index=True)
    
    return amoc_df

In [5]:
#Rare Event Algorithm functions

def resample(ntrajs,traj_list,dt,k,amoc_df,agg_label,agg_run):
    # resampling
    
    # compute weights
    obsint=np.empty(ntrajs)
    for traj in range(ntrajs):
        obsint[traj]=np.sum(traj_list[traj])*dt
    W=np.exp(k*(obsint-np.mean(obsint)))
    # compute the normalizer and the normalized weights
    R=sum(W)/ntrajs
    w=W/R
    
    # compute the number of clones generated by trajectory
    nc=np.empty(ntrajs)
    for traj in range(ntrajs):
        nc[traj]=math.floor(w[traj]+np.random.uniform(0,1))
    
    # compute total number of clones and the difference with original ensemble size 
    Nc=sum(nc)
    dN=Nc-ntrajs
    
    #logic for keeping clones proportional to ancestors
    label=np.arange(ntrajs)
    if dN<0:
        traj2clone=np.empty(int(-dN))
        for traj in range(int(-dN)):    
            traj2clone[traj]=np.random.choice(label,p=nc/sum(nc))  
        for traj in range(int(-dN)):
            nc[int(traj2clone[traj])]=nc[int(traj2clone[traj])]+1
    
    if dN>0:
        traj2kill=np.empty(int(dN))
        for traj in range(int(dN)):
            traj2kill[traj]=np.random.choice(label,p=nc/sum(nc))
            nc[int(traj2kill[traj])]=nc[int(traj2kill[traj])]-1

    killedlabel=np.empty(ntrajs)
    for traj in range(ntrajs):
        if nc[traj]>0:
            killedlabel[traj]=0
        else:
            killedlabel[traj]=1
    
    #assign initialization labels to clones
    initlabel=np.arange(ntrajs)
    udkilledlabel=np.empty(ntrajs)
    udkilledlabel[:]=killedlabel[:]
    for traj in range(ntrajs):
        if nc[traj]>1:
            for n in range(int(nc[traj])-1):
                traj2substitute=np.random.choice(label,p=udkilledlabel/sum(udkilledlabel))
                initlabel[int(traj2substitute)]=traj
                udkilledlabel[int(traj2substitute)]=0

    zero_initials = amoc_df['x0,y0,z0,T0,S0'].unique()
    #print('Unique initials: ',T0_S0)
    #create df for metadata
    metadata = pd.DataFrame(data={'Initial S,T': zero_initials,'Last AMOC value':last_amocs,'Corresponding T value': last_temps,
                                  'Corresponding S Value': last_salts, 'Corresponding x value': last_xes, 
                                  'Corresponding y value': last_yes, 'Corresponding z value': last_zes,
                                  'Label':label,'Exponential':W,'Weight':w,'Number of Clones':nc,'Killed?':killedlabel,
                                  'Init Label':initlabel,'Agg Label':agg_label})
    
    #add perturbation by determining if the traj has been killed, if it has add the perturbation to the temp column
    
    #get killed trajectories to add perturbation to
    killed=metadata['Killed?'].tolist()
    init_temps,init_salts,init_xes,init_yes,init_zes = [],[],[],[],[]
    
    for kill in range(len(killed)):
        if (killed[kill]==1.0):
            #change the last temps and salinities to clone
            init_label = metadata['Init Label'].values[kill]
            mother_filter = metadata[(metadata['Init Label'] == init_label) & (metadata['Killed?'] == 0.0)]
            mother_temp = mother_filter['Corresponding T value'].iloc[0]
            mother_salt = mother_filter['Corresponding S Value'].iloc[0]
            mother_x = mother_filter['Corresponding x value'].iloc[0]
            mother_y = mother_filter['Corresponding y value'].iloc[0]
            mother_z = mother_filter['Corresponding z value'].iloc[0]
            #define noise, amplitude
            epsilon = 10e-6
            r=np.random.uniform(-1,1)
            init_temps.append(mother_temp)
            init_salts.append(mother_salt*(1+epsilon*r))
            init_xes.append(mother_x)
            init_yes.append(mother_y)
            init_zes.append(mother_z)
        else:
            init_temps.append(last_temps[kill])
            init_salts.append(last_salts[kill])
            init_xes.append(last_xes[kill])
            init_yes.append(last_yes[kill])
            init_zes.append(last_zes[kill])
    
    metadata['Agg Label'] = agg_label
    metadata['Initial Temperatures Next Block'] = init_temps
    metadata['Initial Salinities Next Block'] = init_salts
    metadata['Initial x Next Block'] = init_xes
    metadata['Initial y Next Block'] = init_yes
    metadata['Initial z Next Block'] = init_zes

    return metadata,init_xes,init_yes,init_zes,init_temps,init_salts,initlabel,agg_label,killed

In [ ]:
run_metadata = []
run_amoc = []
time=35
ntraj=200
k=4500
agg_label = []
for run in range(100):
    print('Run: ',run)
    if run == 0:
        x_array,y_array,z_array,temp_array,salt_array = get_initials()
        print(len(x_array))
        print("initials gotten")
        amoc_df = simulate_with_resampling(x_array,y_array,z_array,temp_array,salt_array)
        print("Initial simulation done")
    elif (run > 0 and run < time):
        amoc_df = simulate_with_resampling(x_array=init_xes,y_array=init_yes,z_array=init_zes,temp_array=init_temps,
                                           salt_array=init_salts)
    elif (run >= 45 and run < 75):
        k=-10000
        amoc_df = simulate_with_resampling(x_array=init_xes,y_array=init_yes,z_array=init_zes,temp_array=init_temps,
                                           salt_array=init_salts)
    else:
        k=0
        print('K in conditional:',k)
        amoc_df = simulate_with_resampling(x_array=init_xes,y_array=init_yes,z_array=init_zes,temp_array=init_temps,
                                           salt_array=init_salts)
        
    #split into trajectories for input
    #create a unique dataframe for each unique value in S0,T0 column
    unique_starts = amoc_df['x0,y0,z0,T0,S0'].unique()
    traj_dfs,traj_list = [],[]
    last_amocs,last_temps,last_salts,last_xes,last_yes,last_zes = [],[],[],[],[],[]

    #change this to instead of AMOC do T and S forcing
    for start in range(len(unique_starts)):
        trajec = amoc_df.loc[amoc_df['x0,y0,z0,T0,S0']==unique_starts[start]]
        #amoc_list = trajec['AMOC'].tolist()
        salin_list = trajec['S'].tolist()
        traj_dfs.append(trajec)
        #traj_list.append(amoc_list)
        traj_list.append(salin_list)
    
    for traj in traj_dfs:
        #print(traj)
        last_amoc = traj['AMOC'].values[-1]
        last_temp = traj['T'].values[-1]
        last_salt = traj['S'].values[-1]
        last_x = traj['x'].values[-1]
        last_y = traj['y'].values[-1]
        last_z = traj['z'].values[-1]
        last_amocs.append(last_amoc)
        last_temps.append(last_temp)
        last_salts.append(last_salt)
        last_xes.append(last_x)
        last_yes.append(last_y)
        last_zes.append(last_z)
        
    #change labels to reflect ancestors
    agg_run=2
    if (run == 0):
        agg_label=np.arange(ntraj)
    elif (run == 1):
        agg_label=init_for_run0
    else:
        for kill in range(len(killed)):
            if killed[kill]==1.0:
                init_kill = init_for_run0[kill]
                agg_label[kill]=agg_for_others[init_kill]
        agg_run=-2
        agg_label=agg_label
    
    #try different values of k (4th parameter),change fist param for ntrajs
    metadata,init_xes,init_yes,init_zes,init_temps,init_salts,init_for_run0,agg_for_others,killed = resample(ntraj,traj_list,0.01,
                                                                                                      k,amoc_df,agg_label,agg_run)
    metadata['Run name'] = run
    metadata['k'] = k
    print('k after adding to metadata:',k)
    amoc_df['Run name'] = run
    amoc_df['k'] = k
    
    
    run_metadata.append(metadata)   
    run_amoc.append(amoc_df)

Run:  0
200
initials gotten
Default Gottwald parameters:
[0.25, 4.0, 7.5, 0.34, 0.0003, 8.0, 0.1, 1.0, 0.0, 1.0, 0.0195, 0.9, 0.00934, 1.0147, 1.7463]
X0:  1.5301895162245174  Trajectory:  0
X0:  0.1587183440345203  Trajectory:  1
X0:  0.5139467635040706  Trajectory:  2
X0:  1.2616582086115986  Trajectory:  3
X0:  1.4961455137940405  Trajectory:  4
X0:  0.7507531643499533  Trajectory:  5
X0:  0.2039312695363245  Trajectory:  6
X0:  1.0264091364450445  Trajectory:  7
X0:  0.828481144178616  Trajectory:  8


In [ ]:
metadatas = pd.concat(run_metadata,ignore_index=True)
amocs = pd.concat(run_amoc,ignore_index=True)
metadatas.head()

In [10]:
metadatas.head()

,"Initial S,T",Last AMOC value,Corresponding T value,Corresponding S Value,Corresponding x value,Corresponding y value,Corresponding z value,Label,Exponential,Weight,...,Killed?,Init Label,Agg Label,Initial Temperatures Next Block,Initial Salinities Next Block,Initial x Next Block,Initial y Next Block,Initial z Next Block,Run name,k
0,"1.5301895162245174,-0.7504815730047827,1.85865...",0.157029,0.570052,0.413024,0.777139,-0.354370,-0.630325,0,4.326685e-11,1.624932e-33,...,1.0,50,0,0.584111,0.435419,0.917596,-0.187894,-0.534380,0,2967
1,"0.1587183440345203,1.2652813035197252,-0.10267...",0.153470,0.577124,0.423654,0.583980,-0.773374,-1.269493,1,6.014569e-03,2.258834e-25,...,1.0,6,1,0.578734,0.437314,0.932876,1.294934,-0.026908,0,2967
2,"0.5139467635040706,0.4500435851437793,1.151405...",0.159216,0.579685,0.420469,0.368049,-0.416038,1.559534,2,4.023524e-04,1.511076e-26,...,1.0,6,2,0.578737,0.437314,0.932876,1.294934,-0.026908,0,2967
3,"1.2616582086115986,1.1432023423021749,0.489161...",0.155064,0.579176,0.424112,1.862599,-0.114031,-1.125728,3,1.572223e+07,5.904646e-16,...,1.0,132,3,0.586780,0.438775,0.233704,0.504377,1.046725,0,2967
4,"1.4961455137940405,-1.4216860585754605,-0.1096...",0.165024,0.581671,0.416648,1.606163,-1.038717,0.957108,4,1.168003e-04,4.386556e-27,...,1.0,148,4,0.582612,0.443652,0.600735,0.433588,1.461497,0,2967


In [16]:
#metadatas.to_csv(f"shifted/metadata_{time}y_{ntraj}traj_{k}k.csv",index=False)
#new folders to save T and S forcing
metadatas.to_csv(f"amocs/salt_{time}y_{ntraj}traj_{k}k.csv",index=False)
metadatas.to_csv(f"salts/salt_{time}y_{ntraj}traj_{k}k.csv",index=False)